In [33]:
# prepare packages
import requests, pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import logging
# from pythonjsonlogger import jsonlogger
import logmatic
import os
import time, random

In [24]:

# 設定現在時間 以時間來作為log檔名 一個log 以日計算
# 並在當下目錄創建 log folder
def get_time():
    now = datetime.utcnow()
    path = "./log"

    utc_8 = timezone(timedelta(hours=8))
    now = now.replace(tzinfo=timezone.utc)

    time_now = now.astimezone(utc_8)

    split = str(time_now).split(" ")
    file_date = split[0]
    split = split[1].split(":")
    file_time = split[0]+split[1]
    if not os.path.exists(path) :
        os.makedirs(path)
        
    return path + "/" + file_date 




In [25]:
# memory
url = "https://www.ptt.cc/bbs/Food/index2.html"

# set proxy to use
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [26]:
# example use of logmatic 
# more simple 

logger = logging.getLogger()
 
handler = logging.FileHandler(get_time())
handler.setFormatter(logmatic.JsonFormatter())
 
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [27]:
# 連線流程
def get_connect(url):
    i = 0
    try :
        resp = requests.get(url,headers=header,proxies=proxylist[i])
        resp.encoding = "utf-8"
        # 設定連線狀況 log
        code = "HTTP response code = " + str(resp.status_code)
        logger.info(code)
        while resp.status_code != 200 :
            resp.close()
            i = (i + 1) % 6
            resp = requests.get(url,headers=header,proxies=proxylist[i])
            resp.encoding = "utf-8"



    except Exception as e:
        logger.error(e)
    return resp.text
    
    

In [28]:
# 進入ptt列表 抓取 標題、推文數、網址 
result = []
soup = BeautifulSoup(get_connect(url))
print("進入ptt : ",url)
post = soup.find_all("div", class_="r-ent")
for r in post:
            # 取得推的數量
    push = 0
    if r.find("div", class_="nrec").text:
        try:
            push = int(r.find("div", class_="nrec").text)
        except ValueError:
                pass
    if r.find("a"):
        # 取得網址
        href = "http://www.ptt.cc" + r.find("a")["href"]
        # 取得標題
        title = r.find("a").text
        
        # 進入第二層 進入每一篇
        # 去除公告 請益 為分類的文章
        if not "公告" in title :
            if not "請益" in title:
                print("現在處理: ", href)
                art = BeautifulSoup(get_connect(href))
                article = []
                content = art.find("div", id="main-content")
                # 保存要丟棄的資訊
                val = content.find_all("span", {"class":"article-meta-value"})
                author = val[0].text
                title_in_content = val[2].text
                date_in_content = val[3].text
                
                # 開始丟棄資訊
                removes = content.find_all("div", class_="article-metaline")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="article-metaline-right")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("span", class_="f2")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="push")
                for remove in removes:
                    remove.extract()
                article.append({
                    "author" : author,
                    "title_in_content" : title_in_content,
                    "date_in_content" : date_in_content,
                    "content" : content.text
                })
                
    result.append({
        "title" : title,
        "push" : push,
        "href" : href,
        "content" : article
        
    })
    
    time.sleep(random.randint(1,6))
                






進入ptt :  https://www.ptt.cc/bbs/Food/index2.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1075984738.A.940.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076050484.A.E4D.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076076457.A.E23.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076140207.A.43C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076164201.A.3B2.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076171700.A.C9E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076174761.A.4F7.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076223828.A.358.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076238985.A.B51.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076244287.A.580.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076261639.A.681.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076297742.A.630.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076391807.A.B79.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076393080.A.4CA.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076404948.A.EAA.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1076425294.A.ECB.html
現在處理:  http://www.ptt.c

NameError: name 'resp' is not defined

In [76]:
import json
# 轉成 Dataframe
df = pd.DataFrame(result)



In [88]:
# 排列欄位順序

df = df[ ["title", "push", "href", "content" ] ]

In [91]:
df.iloc[1]

title                                                杜樂麗法國茶館
push                                                       0
href       http://www.ptt.cc/bbs/Food/M.1076050484.A.E4D....
content    [{'author': 'mimique41 (here for u)', 'title_i...
Name: 1, dtype: object

In [ ]:
# example use of json_logging
# more detail
import logging
import sys

import json_logging

# log is initialized without a web framework name
json_logging.ENABLE_JSON_LOGGING = True
json_logging.init()

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

#logger.info("test log statement")
#logger.info("test log statement", extra={'props': {"extra_property": 'extra_value'}})

In [ ]:
# set logging 
# 設定 log config 
# less detail

handler = logging.FileHandler(get_time())  # Or FileHandler or anything else
# Configure the fields to include in the JSON output. message is the main log string itself
format_str = '%(message)%(levelname)%(name)%(asctime)'
formatter = jsonlogger.JsonFormatter(format_str)
handler.setFormatter(formatter)
logger = logging.getLogger(__name__)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
# Normally we would attach the handler to the root logger, and this would be unnecessary
logger.propagate = False
